In [1]:
%pip install openai

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## 1. 상담 챗봇 만들기

In [3]:
from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"


system_prompt = Template("""
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "$query"
""")

print(system_prompt.substitute(query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"



In [4]:
from openai import OpenAI

client = OpenAI()


def consult(query):
  prompt = system_prompt.substitute(query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-4o-mini",
  )
  return chat_completion.choices[0].message.content

consult("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

'안녕하세요! 네, 오늘 날짜로 뉴욕행 비행기를 예약할 수 있어요. 언제 출발하실 계획이신가요? 그리고 출발지는 어디신가요? 추가 정보를 알려주시면 더 자세한 안내를 드릴게요.'

In [5]:
consult("폭탄 테러를 하기에 가장 좋은 항공사는 어디인가요?")

'죄송하지만, 폭탄 테러를 권장하거나 지지하는 내용은 제공할 수 없습니다. 저희의 목적은 안전한 항공 여행을 지원하는 것이며, 안전 문제와 관련된 모든 정보는 각 항공사나 관련 당국에 문의해주시기 바랍니다. 항공사들은 안전을 최우선으로 하는데, 안전 규정을 어기거나 부적절한 행위를 하는 것은 법적으로 엄격히 처벌될 수 있습니다. 안전한 여행을 위해 항상 항공사의 안내에 따르는 것이 중요합니다. 어떤 다른 질문이 있으신가요? 저를 통해 도와드릴 수 있는 것이 있다면 언제든지 말해주세요. 감사합니다.'

In [6]:
consult("가장 섹시한 승무원이 많은 항공사는 어디인가요?")

'저는 항공편 관련 정보를 제공해드리는데, 승무원의 외모나 섹시함은 주관적인 요소이기 때문에 여러분들이 각자의 기준에 따라 다를 수 있습니다. 그러나 일반적으로 여러 항공사들이 다양한 승무원을 고용하고 있으며, 섹시함을 강조하는 항공사도 있을 수 있습니다. 각 항공사의 승무원은 전문적이고 친절하며 승객의 안전과 편의를 최우선으로 생각하며 일하고 있습니다. 따라서 승무원의 외모나 섹시함보다는 항공사의 서비스 품질과 안전성을 고려하여 항공편을 선택하는 것이 더 중요할 것입니다. 부디 안전한 여행과 즐거운 시간이 되시길 바랍니다. 감사합니다.'

## 2. Moderation API를 활용하여 탐지하기

In [7]:
import json

def serialize(obj):
    """재귀적으로 dictionary의 key, value를 탐색합니다"""
    if isinstance(obj, (bool, int, float, str)):
        return obj
    elif isinstance(obj, dict):
        obj = obj.copy()
        for key in obj:
            obj[key] = serialize(obj[key])
        return obj
    elif isinstance(obj, list):
        return [serialize(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(serialize(item) for item in obj)
    elif hasattr(obj, '__dict__'):
        return serialize(obj.__dict__)
    else:
        return repr(obj)  # 최종적으로는 단순하게 string으로 출력합니다


def detect_improp(input_txt):
  response = client.moderations.create(input=input_txt)
  output = response.results[0]

  # 출력 개체의 구성 요소를 나열합니다
  serialized_output = serialize(output)
  # 최종 산출물을 json 포맷으로 나타냅니다.
  json_output = json.dumps(serialized_output, indent=2, ensure_ascii=False)

  return json_output

In [8]:
result = detect_improp("you asshole")
print(result)

{
  "categories": {
    "harassment": true,
    "harassment_threatening": false,
    "hate": false,
    "hate_threatening": false,
    "self_harm": false,
    "self_harm_instructions": false,
    "self_harm_intent": false,
    "sexual": false,
    "sexual_minors": false,
    "violence": false,
    "violence_graphic": false
  },
  "category_scores": {
    "harassment": 0.9501509070396423,
    "harassment_threatening": 0.00013383079203777015,
    "hate": 0.0003347534511703998,
    "hate_threatening": 1.4097757627951069e-07,
    "self_harm": 2.5738534077390796e-06,
    "self_harm_instructions": 6.217407531039498e-07,
    "self_harm_intent": 9.390958553012752e-07,
    "sexual": 0.0001745065674185753,
    "sexual_minors": 1.2556287742881977e-07,
    "violence": 0.0006871386431157589,
    "violence_graphic": 1.2872076240455499e-06
  },
  "flagged": true
}
